<a href="https://colab.research.google.com/github/yingzibu/MOL2ADMET/blob/main/results/chemberta_ADMET_result.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
print(torch.__version__)
print('cuda: ', torch.cuda.is_available())

2.1.0+cu121
cuda:  True


In [ ]:
! pip install rdkit --quiet
! pip install PyTDC --quiet
! pip install mycolorpy --quiet
! pip install selfies  --quiet
! pip install pubchempy --quiet
! pip install dgllife --quiet
! pip install molvs --quiet
! pip install dgl -f https://data.dgl.ai/wheels/cu121/repo.html --quiet
! pip install dglgo -f https://data.dgl.ai/wheels-test/repo.html --quiet
! pip install DeepPurpose --quiet
! pip install git+https://github.com/bp-kelley/descriptastorus --quiet
! pip install pandas-flavor --quiet


# specific for chemberta
!pip uninstall transformers -y
!pip install transformers==4.30.2 --quiet

In [1]:
cd /content/drive/MyDrive/ADMET

/content/drive/MyDrive/ADMET


In [2]:
from scripts.func_utils import make_path, convert_with_qed_sa, get_min, \
                                plot_loss, plot_performance

from scripts.eval_utils import *
from scripts.preprocess_mols import *
from scripts.model_architecture import *
from scripts.dataset import *
from scripts.train import *
import yaml
import pandas as pd
from scripts.get_vocab import *

from tdc.single_pred import ADME
from tdc.single_pred import Tox
from scripts.CONSTANT import *
from scripts.yaml_utils import *
from scripts.TRAIN import *
from scripts.MUE import *
from IPython.display import Code
print('VOCAB TYPE:', VOCAB_TYPE)

VOCAB TYPE: smiles


In [ ]:
Code('scripts/train.py')

"""
Date: 09/07/2023
Mods: 11/08/2023, edited train_epoch_VAE: add tqdm
Athr: Bu
Aims: train functions for all models
Mods: 
      02/05/2024, add scale_back in model_predict function
Func:
      tbc...
"""

import yaml
import time
import torch
import IPython
import imageio
import numpy as np
import pandas as pd
from os import walk
import torch.nn as nn
from tqdm import tqdm
from torch.nn import Module
import torch.nn.functional as F
from torch.nn.utils import clip_grad_norm_
from dgllife.utils import EarlyStopping, Meter
from torch.utils.data import DataLoader, Dataset


from scripts.CONSTANT import *
from scripts.eval_utils import *
from scripts.func_utils import *
from scripts.dataset import get_loader
from scripts.model_architecture import *

clip_grad  = 50
model_types = ['MLP', 'AttentiveFP', 'GIN', 'RNN', 
                'VAE', 'RNN_pretrain', 'MUE', 'chemBERTa']

def count_bool(lst): return sum(lst)

def count_parameters(model: Module):
    return sum(p.numel() for p in model.parameters())

def count_trainable(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

def get_optim_params(model):
    return (p for p in model.parameters() if p.requires_grad)
    
def initialize_weights(m):
    if hasattr(m, 'weight') and m.weight.dim() > 1:
        nn.init.xavier_uniform_(m.weight.data)

def clamp(number, min_value=1e-5, max_value=1-1e-5):
    return max(min_value, min(number, max_value))
# uncertainty weight
class MTLoss(Module): # calculate multitask loss with trainable parameters
    def __init__(self, task_num, weight_loss=None, device='cuda'): 
        super(MTLoss, self).__init__()
        self.task_num = task_num
        self.device = device
        if weight_loss==None: weight_loss = [1.0] * self.task_num
        # 𝜂 := 𝑙𝑜𝑔𝜎2
        self.eta = nn.Parameter(torch.tensor(weight_loss, device=self.device))
        
    def forward(self, loss_list, IS_R=None):
        assert len(loss_list) == self.task_num
        if IS_R != None: 
            assert len(IS_R) == len(loss_list)
            for i in range(len(IS_R)):
                if IS_R[i] == True: loss_list[i] /= 2 
        loss_list = torch.Tensor(loss_list).to(self.device)
        total_loss = loss_list * torch.exp(-self.eta) + self.eta * 0.5 

        weight_loss = [float(self.eta[i].item()) for i in range(self.task_num)]
        weight_loss = [np.exp(-1.0 * i) for i in weight_loss]
        if IS_R != None:
            for i in range(len(IS_R)):
                if IS_R[i] == True: weight_loss[i] /= 2

        weight_loss = [i/sum(weight_loss) for i in weight_loss]
        
        for n in range(len(weight_loss)): 
            if weight_loss[n] < 1e-5: weight_loss[n] = clamp(weight_loss[n])
        weight_loss = [i/sum(weight_loss) for i in weight_loss]

        
        return total_loss.sum(), weight_loss


def init_model(**config):
    """need incorporate all models here! """
    if   config['model_type'] == 'MLP':          model = Classifier(**config)
    elif config['model_type'] == 'GIN':          model = GIN_MOD(**config) 
    elif config['model_type'] == 'AttentiveFP':  model = AttentiveFP(**config)
    elif config['model_type'] == 'RNN':          model = RNN(**config)
    elif config['model_type'] == 'VAE':          model = RNNVAE(**config)
    elif config['model_type'] == 'RNN_pretrain': model = RNN_pretrain(**config)
    elif config['model_type'] == 'MUE':          model = Classifier(**config)
    elif config['model_type'] == 'chemBERTa':    model = chemBERTa(**config)
    else:                    print('invalid model type:', config['model_type'])
    return model


def get_train_fn(model_type):
    if   model_type == 'VAE':       return train_epoch_VAE
    elif model_type in model_types: return train_epoch_MLP
    else: print('invalid model type:', model_type); return

def get_eval_fn(model_type):
    if   model_type == 'VAE':       return train_epoch_VAE
    elif model_type in model_types: return train_epoch_MLP
    else: 

In [6]:
_=train_mt(names_M5, 'chemBERTa', 'M5/uw', 3, retrain=False, uw=True, batch_size=128, ver_freq=5)
_=train_mt(names_A3, 'chemBERTa', 'A3/uw', 3, retrain=False, uw=True, batch_size=128, ver_freq=5)
_=train_mt(names_T3, 'chemBERTa', 'T3/uw', 3, retrain=False, uw=True, batch_size=128, ver_freq=5)


Run # 0 for chemBERTa MT	 | save dir:  M5/uw/chemBERTa_MT_0 | 	--> pre data loaded

Run # 1 for chemBERTa MT	 | save dir:  M5/uw/chemBERTa_MT_1 | 	--> pre data loaded

Run # 2 for chemBERTa MT	 | save dir:  M5/uw/chemBERTa_MT_2 | 	--> pre data loaded
repeated num # 3 idx 1 has the lowest loss
******************** CYP2C19_Veith ******************** 
	|       acc      |       w_acc      |       prec      |       recall      |       sp      |       f1      |       auc      |       mcc      |       ap      
	&0.803$\pm$0.023  &0.803$\pm$0.025  &0.774$\pm$0.018  &0.806$\pm$0.045  &0.801$\pm$0.015  &0.789$\pm$0.029  &0.878$\pm$0.016  &0.606$\pm$0.048  &0.851$\pm$0.013  
 idx 1: &0.823            &0.823            &0.797            &0.824            &0.822            &0.810            &0.890            &0.644            &0.857            

******************** CYP2D6_Veith ******************** 
	|       acc      |       w_acc      |       prec      |       recall      |       sp      |      

In [ ]:
names_all_test = [
 'AMES',
 'CYP2C9_Substrate_CarbonMangels',
 'CYP2D6_Substrate_CarbonMangels',
 'CYP3A4_Substrate_CarbonMangels',
 'DILI',
 'Skin Reaction',
 'Carcinogens_Lagunin',
 'ClinTox']

In [7]:
names_bad = []
for name in names_all[:-2]:
    try:
        train_mt(name, 'chemBERTa', f'ST/{name}', 3, retrain=False, uw=False)
    except:
        names_bad.append(name)


Run # 0 for chemBERTa ST	 | save dir:  ST/Caco2_Wang/chemBERTa_ST_0 | 	--> pre data loaded

Run # 1 for chemBERTa ST	 | save dir:  ST/Caco2_Wang/chemBERTa_ST_1 | 	--> pre data loaded

Run # 2 for chemBERTa ST	 | save dir:  ST/Caco2_Wang/chemBERTa_ST_2 | 	--> pre data loaded
repeated num # 3 idx 0 has the lowest loss
******************** Caco2_Wang ******************** 
	|       mae      |       mse      |       rmse      |       r2      |       pcc      |       spearman      
	&0.371$\pm$0.010  &0.228$\pm$0.014  &0.477$\pm$0.014  &0.641$\pm$0.021  &0.809$\pm$0.007  &0.748$\pm$0.007  
 idx 0: &0.361            &0.216            &0.465            &0.660            &0.816            &0.758            

******************** Caco2_Wang ******************** 
	|  mae  |  rmse  |  r2  
single: &0.361   &0.465   &0.660   





Run # 0 for chemBERTa ST	 | save dir:  ST/Lipophilicity_AstraZeneca/chemBERTa_ST_0 | 	--> pre data loaded

Run # 1 for chemBERTa ST	 | save dir:  ST/Lipophilicity_AstraZ